In [1]:
import numpy as np,pandas as pd,matplotlib.pyplot as plt, seaborn as sns

In [2]:
df=pd.read_csv("census-income .csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  annual_income   32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [4]:
df['annual_income']

0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
32556    <=50K
32557     >50K
32558    <=50K
32559    <=50K
32560     >50K
Name: annual_income, Length: 32561, dtype: object

In [5]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

In [6]:
df['annual_income']=LabelEncoder().fit_transform(df['annual_income'])

In [7]:
df['annual_income']

0        0
1        0
2        0
3        0
4        0
        ..
32556    0
32557    1
32558    0
32559    0
32560    1
Name: annual_income, Length: 32561, dtype: int32

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load the dataset
data = pd.read_csv('census-income .csv')

# Encode target variable
data['annual_income'] = LabelEncoder().fit_transform(data['annual_income'])

# Separate features and target variable
X = data.drop(columns=['annual_income'])
y = data['annual_income']

In [9]:

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(exclude=['object']).columns

# Define preprocessing steps
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model pipeline with Random Forest
rf_model = RandomForestClassifier(random_state=0)
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', rf_model)])


In [10]:

# # Split data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# # Define parameter grid for GridSearchCV
# param_grid = {
#     'classifier__n_estimators': [50, 100, 200],
#     'classifier__max_depth': [None, 10, 20, 30],
#     'classifier__min_samples_split': [2, 5, 10],
#     'classifier__min_samples_leaf': [1, 2, 4]
# }


In [11]:
#grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)


In [12]:
from sklearn.model_selection import RandomizedSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5],
    'classifier__min_samples_leaf': [1, 2]}

In [13]:
random_search = RandomizedSearchCV(
    pipeline, param_distributions=param_grid, 
    n_iter=20, cv=2, scoring='accuracy', n_jobs=-1, verbose=1, random_state=0
)

# Fit the model
random_search.fit(X_train, y_train)


Fitting 2 folds for each of 20 candidates, totalling 40 fits


RandomizedSearchCV(cv=2,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')),
                                                                              ('cat',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(str...
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object'))])),
                                             ('classifier',
                                              RandomForestClassifier(random_state=0))]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'classifier__max_depth': [None, 10, 20],
                                        'classifier__min_samples_leaf': [1, 2],
                                        'classifier__min_samples_split': [2, 5],
                                        'classifier__n_estimators': [50, 100,
                                                                     200]},
                   random_state=0, scoring='accuracy', verbose=1)

In [14]:

# Best parameters and best score from grid search
best_params = random_search.best_params_
best_score = random_search.best_score_

# Predict and evaluate on test data
y_pred = random_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Best Parameters:", best_params)
print("Best Cross-Validation Accuracy:", best_score)
print("Test Accuracy:", accuracy)
print("Classification Report:\n", report)


Best Parameters: {'classifier__n_estimators': 200, 'classifier__min_samples_split': 2, 'classifier__min_samples_leaf': 2, 'classifier__max_depth': None}
Best Cross-Validation Accuracy: 0.8632908476658476
Test Accuracy: 0.8593582066635959
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.93      0.91      4918
           1       0.76      0.63      0.69      1595

    accuracy                           0.86      6513
   macro avg       0.82      0.78      0.80      6513
weighted avg       0.85      0.86      0.85      6513



In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Load the dataset
data = pd.read_csv('census-income .csv')

# Encode target variable: Convert annual_income to binary 0 and 1
data['annual_income'] = LabelEncoder().fit_transform(data['annual_income'])

# Separate features and target variable
X = data.drop(columns=['annual_income'])
y = data['annual_income']

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(exclude=['object']).columns

# Define preprocessing for numerical and categorical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Preprocess data
X = preprocessor.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Convert labels to categorical format for binary classification
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Build the neural network model
model = Sequential([
    Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
y_true = y_test.argmax(axis=1)

# Classification report
report = classification_report(y_true, y_pred_classes)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Classification Report:\n", report)


C:\Users\shind\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
652/652 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.8025 - loss: 0.4211 - val_accuracy: 0.8468 - val_loss: 0.3213
Epoch 2/20
652/652 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8501 - loss: 0.3237 - val_accuracy: 0.8512 - val_loss: 0.3193
Epoch 3/20
652/652 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8593 - loss: 0.3112 - val_accuracy: 0.8534 - val_loss: 0.3135
Epoch 4/20
652/652 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8596 - loss: 0.3048 - val_accuracy: 0.8522 - val_loss: 0.3134
Epoch 5/20
652/652 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8601 - loss: 0.3077 - val_accuracy: 0.8503 - val_loss: 0.3174
Epoch 6/20
652/652 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8617 - loss: 0.2972 - val_accuracy: 0.8526 - val_loss: 0.3130
Epoch 7/20
652/652 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8700 - loss: 0.2912 - val_accuracy: 0.8518 - val_loss: 0.3130
Epoch 8/20
652/652 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8677 - loss: 0.2987 - val_accuracy: 0.